In [1]:
%load_ext nb_mypy

Version 1.0.2


In [2]:
import sys
from typing import *
from random import random
sys.version

'3.9.7 (default, Sep 16 2021, 08:50:36) \n[Clang 10.0.0 ]'

In [3]:
def randoms(num: int) -> list[float]:
    if num >= 0:
        return [random() for _ in range(num)]
    else:
        return None # not okay, NoneType is not list[float]

<cell>5: error: Incompatible return value type (got "None", expected "List[float]")


In [4]:
def randoms(num: int) -> Optional[list[float]]:
    if num >= 0:
        return [random() for _ in range(num)]
    else:
        return None # okay, return type is Optional[...]

In [5]:
# Note that None as a type hint is a special case and is replaced by type(None).

def randoms(num: int) -> Union[list[float], None]:
    if num >= 0:
        return [random() for _ in range(num)]
    else:
        return None # okay, return type is Union[..., None]

In [6]:
def randoms(num: int) -> list[float]:
    if num >= 0:
        return [random() for _ in range(num)]
    else:
        raise ValueError # okay, there is no typed way to communicate raised exceptions

In [7]:
def randoms(num: int) -> Union[float, list[float], None]:
    if num == 1:
        return random()                       # float
    elif num >= 0:
        return [random() for _ in range(num)] # list[float]
    else:
        return None                           # None

In [8]:
def randoms(num: int) -> float | list[float] | None: # does not work in Python 3.9, only 3.10
    if num == 1:
        return random()
    if num >= 0:
        return [random() for _ in range(num)]
    else:
        return None

<cell>1: error: X | Y syntax for unions requires Python 3.10


TypeError: unsupported operand type(s) for |: 'type' and 'types.GenericAlias'

In [9]:
FeatureVector = list[float] # type alias

def predict(fv: FeatureVector) -> float:
    return random()

fv: list[float] = [0.1, 0.2, 0.3]
predict(fv) # okay
fv: FeatureVector = [0.1, 0.2, 0.3]
predict(fv) # okay

0.5108025836830542

In [10]:
FeatureVector = NewType('FeatureVector', list[float])
# all FeatureVectors are list[float], but not all list[float] are FeatureVectors

def predict(fv: FeatureVector) -> float:
    return random()

fv: list[float] = [0.1, 0.2, 0.3]
predict(fv) # not okay
fv: FeatureVector = FeatureVector([0.1, 0.2, 0.3]) # explicit cast
predict(fv) # okay

<cell>8: error: Argument 1 to "predict" has incompatible type "List[float]"; expected "FeatureVector"


0.7407515877106208

In [11]:
T = TypeVar('T') # declare type variable T to be used
def first(li: list[T]) -> T:
    return li[0] # okay

In [12]:
T = TypeVar('T') # declare type variable T to be used
def first(li: list[T]) -> Optional[T]:
    return li[0] if len(li) > 0 else None # okay

In [13]:
T = TypeVar('T') # declare type variable T to be used
def first(li: list[T]) -> T:
    return "hello" # not okay

<cell>3: error: Incompatible return value type (got "str", expected "T")


In [14]:
T = TypeVar('T') # declare type variable T to be used
def add(a: T, b: T) -> T:
    return a+b # checks for __add__()

<cell>3: error: Unsupported left operand type for + ("T")


In [15]:
class Addable(Protocol):
  def __add__(self, other):
    raise NotImplementedError

In [16]:
def add(a: Addable, b: Addable) -> Addable:
    print(type(a), type(b))
    return a+b # checks for __add__()

add(str(1), str(2))   # okay
add(int(1), int(2))   # okay
add(int(1), float(2)) # okay
add(int(1), str(2))   # not a type error, but a run-time error

<class 'str'> <class 'str'>
<class 'int'> <class 'int'>
<class 'int'> <class 'float'>
<class 'int'> <class 'str'>


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [17]:
T = TypeVar('T', bound=Addable)
    
def add(a: T, b: T) -> T:
    print(type(a), type(b))
    return a+b # checks for __add__(..)

add(str(1), str(2))   # okay
add(int(1), int(2))   # okay
add(int(1), float(2)) # okay
add(int(1), str(2))   # type error and run-time error

<cell>10: error: Value of type variable "T" of "add" cannot be "object"


<class 'str'> <class 'str'>
<class 'int'> <class 'int'>
<class 'int'> <class 'float'>
<class 'int'> <class 'str'>


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [19]:
T = TypeVar('T', int, float, str)

def add(a: T, b: T) -> T:
    return a+b # checks for __add__()

add(int(1), float(2)) # okay

3.0